# Algoritmos genéticos

In [69]:
import random
import bisect
# Needed to hide warnings in the matplotlib sections
import warnings
#from numpy import *
warnings.filterwarnings("ignore")

## Clases y funciones aima-python

In [70]:
class Grafo:
    """Un Grafo es un array bidimensional de tamño N*N, siendo N el número de ciudades,
    representadas por los valores 0,...,N-1; 0 se toma como ciudad de partida.
    El valor de la posición (i,j) es la distancia entre las ciudades i y j,
     que es el mismo que la distancia entre j e i """
    def __init__(self,lista):
        "N es el número de ciudades correspondiente a la lista de valores de una instancia en la sintaxis EDGE_WEIGHT_TYPE"
        self.N = int(((8*len(lista)+1)**0.5)-1)/2
        #print(self.N)
        self.ciudades = list(range(int(self.N)))
        self.Dist = [[0]*int(self.N) for i in self.ciudades]
        x_acc = 0
        for x in range(int(self.N)):
            x_acc += x
            for y in range(x+1):
                self.Dist[x][y] = self.Dist[y][x] = lista[x_acc+y]

In [71]:
def genetic_algorithm_stepwise(population, fitness_fn, gene_pool, ngen=1200, pmut=0.1):
    for generation in range(int(ngen)):
        # Elitism may be here - ADDED
        previous_best = max(population, key=fitness_fn)
        population = [mutate2(order_crossover(*select(2, population, fitness_fn)), pmut) for i in range(len(population)-1)]
        population.append(previous_best)
        # stores the individual genome with the highest fitness in the current population
        current_best = max(population, key=fitness_fn)
        #print(f'Current best: {current_best}\t\tGeneration: {str(generation)}\t\tFitness: {fitness_fn(current_best)}\r', end='')
        #print(f'Current best: {current_best}\t\tGeneration: {str(generation)}\t\tCost: {1/fitness_fn(current_best)}\t\tFitness: {fitness_fn(current_best)}\r')
        print(f"{1/fitness_fn(current_best)},{fitness_fn(current_best)}")
    return max(population, key=fitness_fn)       

def init_population(pop_number, gene_pool):
    # a chromosome is a random permutation of the alphabet
    population = []
    for _ in range(pop_number):
        # Shuffle the gene pool and take the first pool_size elements as an individual
        v = gene_pool[:]
        random.shuffle(v)
        population.append(v)
    return population

def select(r, population, fitness_fn):
    fitnesses = list(map(fitness_fn, population))
    value_range = max(fitnesses) - min(fitnesses)
    scaled_fitnesses = [(x - min(fitnesses)) / value_range for x in fitnesses]
    sampler = weighted_sampler(population, scaled_fitnesses)
    return [sampler() for i in range(r)]

def weighted_sampler(seq, weights):
    """Return a random-sample function that picks from seq weighted by weights."""
    totals = []
    for w in weights:
        totals.append(w + totals[-1] if totals else w)
    return lambda: seq[bisect.bisect(totals, random.uniform(0, totals[-1]))]
    # bisect(a,x) -> insertion position of a in a sorted list x - AL REVES

def uniform_crossover(x, y):
    n = 0
    child = [-1] * N
    indexes = [0] * N
    # de x se copian los valores de las posiciones con indexex[i] == 1 en las mismas posiciones en child
    for i in  range(N):
        indexes[i] = random.randint(0,1) 
        if indexes[i] == 1:
            child[i] = x[i]
            n += 1
    # El resto (N-n) se copia de y en su orden relativo, desde el principio
    i = 0 # indice en y
    k = 0 # indice en child
    for t in range(N-n):
        while y[i] in child[:]:
            i += 1
        while child[k] != -1:
            k += 1
        child[k] = y[i]
        i += 1   
    return child

def order_crossover(x: list, y: list):
    cut1, cut2 = sorted(random.sample(range(len(x)), 2))
    result = [None] * len(x)
    result[cut1:cut2 + 1] = x[cut1:cut2 + 1]
    index = cut2 + 1
    index %= len(x)
    for element in y:
        if element not in result:
            result[index] = element
            index += 1
            index %= len(x)
    return result

def mutate2(x, pmut):
    if random.uniform(0, 1) >= pmut:
        return x
    i, j = random.sample(range(N), 2)
    x[i], x[j] = x[j], x[i]
    return x

## Instancias

In [72]:
gr17 = [
    0, 
    633, 0, 
    257, 390, 0, 
    91, 661, 228, 0, 
    412, 227, 169, 383, 0, 
    150, 488, 112, 120, 267, 0, 
    80, 572, 196, 77, 351, 63, 0, 
    134, 530, 154, 105, 309, 34, 29, 0, 
    259, 555, 372, 175, 338, 264, 232, 249, 0, 
    505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 
    353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 
    324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0, 
    70, 567, 191, 27, 346, 83, 47, 68, 189, 439, 287, 254, 0, 
    211, 466, 74, 182, 243, 105, 150, 108, 326, 336, 184, 391, 145, 0, 
    268, 420, 53, 239, 199, 123, 207, 165, 383, 240, 140, 448, 202, 57, 0, 
    246, 745, 472, 237, 528, 364, 332, 349, 202, 685, 542, 157, 289, 426, 483, 0, 
    121, 518, 142, 84, 297, 35, 29, 36, 236, 390, 238, 301, 55, 96, 153, 336, 0,
 ]
grafo = Grafo(gr17)

## Resolución del problema con GA

In [73]:
max_population = 100 # Definimos la población máxima
mutation_rate = 0.07 # 7%
max_generations = 100
N = 16 # Number of cities - 1
gene_pool: list[int] = list(range(N)) # Create the gene pool from 0 to N-2
gene_pool = [x+1 for x in gene_pool] # We add 1 in order to have from 1 to N-1 cities, leaving the 0 as the initial city
print(gene_pool)
population: list[list[int]] = init_population(100, gene_pool) # Create the population
print(population[:5])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
[[6, 3, 8, 15, 1, 5, 7, 2, 11, 12, 9, 10, 4, 16, 14, 13], [2, 12, 10, 3, 7, 13, 1, 9, 6, 4, 5, 8, 15, 14, 16, 11], [4, 14, 2, 8, 6, 16, 13, 10, 1, 12, 9, 3, 11, 15, 7, 5], [5, 2, 16, 9, 3, 14, 10, 15, 13, 11, 6, 7, 4, 1, 8, 12], [11, 1, 2, 7, 5, 13, 16, 4, 12, 9, 6, 15, 14, 10, 8, 3]]


In [74]:
def fitness(q):
    coste = 0
    lastCity = 0
    for city in q:
        coste = coste + grafo.Dist[lastCity][city]
        lastCity = city
    coste = coste + grafo.Dist[lastCity][0]
    return 1/coste

In [75]:
import time
start_t = time.time()
print("cost,fitness")
solution = genetic_algorithm_stepwise(population, fitness, gene_pool, ngen=5000)
solution.insert(0,0)
end_t = time.time()
#print(solution)
#print(f"Took: {end_t - start_t}")

cost,fitness
3590.0,0.0002785515320334262
3511.0,0.0002848191398461977
3184.0,0.000314070351758794
3184.0,0.000314070351758794
3100.0,0.0003225806451612903
2752.0,0.0003633720930232558
2752.0,0.0003633720930232558
2748.0,0.000363901018922853
2748.0,0.000363901018922853
2748.0,0.000363901018922853
2748.0,0.000363901018922853
2748.0,0.000363901018922853
2732.0,0.00036603221083455345
2732.0,0.00036603221083455345
2732.0,0.00036603221083455345
2732.0,0.00036603221083455345
2732.0,0.00036603221083455345
2732.0,0.00036603221083455345
2732.0,0.00036603221083455345
2732.0,0.00036603221083455345
2732.0,0.00036603221083455345
2732.0,0.00036603221083455345
2732.0,0.00036603221083455345
2732.0,0.00036603221083455345
2732.0,0.00036603221083455345
2682.0,0.0003728560775540641
2437.0,0.0004103405826836274
2437.0,0.0004103405826836274
2437.0,0.0004103405826836274
2437.0,0.0004103405826836274
2437.0,0.0004103405826836274
2437.0,0.0004103405826836274
2437.0,0.0004103405826836274
2437.0,0.000410340582683